In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm

current_directory = os.getcwd()
directory = ""
if current_directory[9:14] == 'vgwis':
    directory = "C:/Users/vgwis/Documents/Bana780"
else:
    directory = "C:/Users/vgwcto/Documents/Python/Bana780"

#for cleaning of PayDelay, LengthOfStay (NaN to zero), DSFS (NaN to zero), DSFS_drug, DSFS_lab, AgeAtFirstClaim, CharlsonIndex

mapping = {
    '162+': 162,
    '5+': 5,
    '7+': 7,
    '10+': 10,
    '1 day': 1,
    '2 days': 2,
    '3 days': 3,
    '4 days': 4,
    '5 days': 5,
    '6 days': 6,
    '1- 2 weeks': 10,
    '2- 4 weeks': 21,
    '4- 8 weeks': 42,
    '0- 1 month': 30,
    '1- 2 months': 60,
    '2- 3 months': 90,
    '3- 4 months': 120,
    '4- 5 months': 150,
    '5- 6 months': 180,
    '6- 7 months': 210,
    '7- 8 months': 240,
    '8- 9 months': 270,
    '9-10 months': 300,
    '10-11 months': 330,
    '11-12 months': 360,
    '0-9': 5,
    '10-19': 15,
    '20-29': 25,
    '30-39': 35,
    '40-49': 45,
    '50-59': 55,
    '60-69': 65,
    '70-79': 75,
    '80+': 85,
    '1-2': 1.5,
    '3-4': 3.5,
}

#Members
members = pd.read_csv(os.path.join(directory, "Members.csv")).drop(columns=['Sex'])

for y in members.columns:
    members[y] = members[y].apply(lambda x: mapping[x] if x in mapping else x)
    members[y] = members[y].fillna(0)
    members[y] = pd.to_numeric(members[y])

members_agg = members.groupby('MemberID').agg({
    'AgeAtFirstClaim': 'mean'
}).reset_index()

#Claims
claims = pd.read_csv(os.path.join(directory, "Claims.csv"))
claims['PayDelay'] = claims['PayDelay']#.apply(lambda x: 162 if x == '162+' else x)
claims = claims[claims['Year'] == 'Y1'].drop(columns=['Vendor','PCP','Year','SupLOS','ProviderID'])

for y in claims.columns:
    claims[y] = claims[y].apply(lambda x: mapping[x] if x in mapping else x)
    if y == 'DSFS' or y == 'LengthOfStay':
            claims[y] = claims[y].fillna(0)
    elif y == 'CharlsonIndex' or y == 'PayDelay':
        claims[y] = pd.to_numeric(claims[y])
    else:
        claims[y] = claims[y].fillna("No Data")

claims_agg = claims.groupby('MemberID').agg({
    'Specialty': 'count',
    'PlaceSvc': 'count',
    'PrimaryConditionGroup': 'count',
    'ProcedureGroup': 'count',
    'CharlsonIndex': 'mean',
    'DSFS': 'mean',
    'PayDelay': 'max',
    'LengthOfStay': 'sum'
}).reset_index()

#Drug
drug = pd.read_csv(os.path.join(directory, "DrugCount.csv")).rename(columns={'DSFS': 'DSFS_drug'})
drug = drug[drug['Year'] == 'Y1'].drop(columns=['Year'])

for y in drug.columns:
    drug[y] = drug[y].apply(lambda x: mapping[x] if x in mapping else x)
    drug[y] = drug[y].fillna(0)
    drug[y] = pd.to_numeric(drug[y])

drug_agg = drug.groupby('MemberID').agg({
    'DrugCount': 'sum',
    'DSFS_drug': 'mean'
}).reset_index()

#Lab
lab = pd.read_csv(os.path.join(directory, "LabCount.csv")).rename(columns={'DSFS': 'DSFS_lab'})
lab = lab[lab['Year'] == 'Y1'].drop(columns=['Year'])

for y in lab.columns:
    lab[y] = lab[y].apply(lambda x: mapping[x] if x in mapping else x)
    lab[y] = lab[y].fillna(0)
    lab[y] = pd.to_numeric(lab[y])

lab_agg = lab.groupby('MemberID').agg({
    'LabCount': 'sum',
    'DSFS_lab': 'mean'
}).reset_index()


merged_df = pd.merge(claims_agg, members_agg, on='MemberID', how='inner'); merged_df = pd.merge(merged_df, drug_agg, on='MemberID', how='inner'); final_df = pd.merge(merged_df, lab_agg, on='MemberID', how='inner')
final_df = final_df.drop(columns = ['PlaceSvc', 'PrimaryConditionGroup', 'ProcedureGroup']).rename(columns = {'Specialty':'NumberOfClaims'})

final_df['CharlsonIndex'] = final_df['CharlsonIndex'].round(1)
final_df['AgeAtFirstClaim'].round(0)
final_df['DSFS'] = final_df['DSFS'].round(1)
final_df['PayDelay'] = final_df['PayDelay'].round(0)
final_df['LengthOfStay'] = final_df['LengthOfStay'].round(0)
final_df['DrugCount'] = final_df['DrugCount'].round(0)
final_df['DSFS_drug'] = final_df['DSFS_drug'].round(0)
final_df['LabCount'] = final_df['LabCount'].round(0)
final_df['DSFS_lab'] = final_df['DSFS_lab'].round(0)

final_df['Score'] = final_df['LengthOfStay'] + final_df['NumberOfClaims'] + final_df['CharlsonIndex'] + final_df['PayDelay']/100 + (final_df['AgeAtFirstClaim']/200)**2 + (final_df['DSFS'] + final_df['DSFS_drug'] + final_df['DSFS_lab'])/3 + (final_df['DrugCount'] + final_df['LabCount'])/2 
final_df['Score'] = final_df['Score'].round(0)
final_df['Score'] = pd.to_numeric(final_df['Score'], downcast='integer')
final_df['Score'] = final_df['Score'].apply(lambda x: 269 if x > 269 else x)
scalar = 0.05204460966 #95th percentile scaled down to 14 days
final_df['DIH_Predict'] = pd.to_numeric(final_df['Score']*scalar, downcast='integer').round(0)

pd.set_option('display.max_rows', None)
print(final_df.head(n=25))
pd.reset_option('display.max_rows', None)

days_in_hosp = pd.read_csv("C:/Users/vgwis/Documents/Bana780/DaysInHospital_Y2.csv")
days_in_hosp['DaysInHospital'] = days_in_hosp['DaysInHospital'].fillna(0)
print(days_in_hosp['DaysInHospital'].describe())

stats_df = pd.merge(final_df, days_in_hosp, on='MemberID', how='right')

x = stats_df['DaysInHospital']
y = stats_df['DIH_Predict']
x = sm.add_constant(x)

model = sm.OLS(y, x).fit()

print(model.summary())

correlation = stats_df['DaysInHospital'].corr(stats_df['DIH_Predict'])
print(f"Correlation between Days in Hospital and Model: {correlation}")